### Tagging - Interpretando dados com funções

In [1]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

texto = "Que pasa pendejo"

class Sentimento(BaseModel):
    """Informa o sentimento e a lingua da frase"""
    sentimento: str = Field(description="sentimento da frase. Deve ser 'positivo', 'negativo' ou 'neutro'")
    lingua: str = Field(description="lingua ou idioma da frase")

tool_sentimento = convert_to_openai_function(Sentimento)
chat = ChatOpenAI(model="gpt-3.5-turbo").bind(functions=[tool_sentimento])

prompt = ChatPromptTemplate.from_messages([
    ("system", "Pense com cuidado para categorizar o texto conforme a s instruções"),
    ("user", "{input}")
])

chain = prompt | chat
chain.invoke({"input": "I dont like this shit"})

c:\Users\johns\OneDrive\Documentos\Projetos\GitHub\Regdoor_Project\venv\Lib\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentimento":"negativo","lingua":"en"}', 'name': 'Sentimento'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 111, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-3b46dad2-9ef3-4dfe-ae56-3fb3dca0c729-0', usage_metadata={'input_tokens': 111, 'output_tokens': 23, 'total_tokens': 134, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### Parseando a saida para obtermos o que queremos

In [2]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (prompt | chat.bind_functions([tool_sentimento], function_call={"name":"Sentimento"}) | JsonOutputFunctionsParser())
chain.invoke({"input": "Qual a temperatura atual em São Paulo?"})

C:\Users\johns\AppData\Local\Temp\ipykernel_8744\878602124.py:3: LangChainDeprecationWarning: The method `BaseChatOpenAI.bind_functions` was deprecated in langchain-openai 0.2.1 and will be removed in 1.0.0. Use :meth:`~langchain_openai.chat_models.base.ChatOpenAI.bind_tools` instead.
  chain = (prompt | chat.bind_functions([tool_sentimento], function_call={"name":"Sentimento"}) | JsonOutputFunctionsParser())


{'sentimento': 'neutro', 'lingua': 'português'}

#### Um exemplo mais interessante de classificação

In [3]:
emails = [
    "Olá, gostaríamos de solicitar um orçamento para uma campanha de tráfego pago no Instagram e Facebook. Poderiam nos enviar uma proposta com valores e prazos? Aguardo retorno. Obrigado!",
    "Oi equipe, estamos com algumas dúvidas sobre o planejamento de conteúdo para o próximo mês. Poderiam agendar uma reunião para alinharmos os detalhes? Obrigado!",
    "Olá, represento a empresa XYZ e estamos em busca de uma agência para produzir vídeos institucionais. Gostaria de saber mais sobre o portfólio de vocês e os valores para esse tipo de serviço. Aguardo retorno.",
    "🔥 Oferta imperdível! Ganhe 10.000 seguidores em minutos. Aumente sua popularidade agora! Com nossa técnica revolucionária, você pode ganhar 10.000 seguidores reais em minutos. Não perca essa chance única! Clique aqui e aproveite!",
    "Prezados, segue em anexo o relatório de desempenho das campanhas do último mês. Qualquer dúvida, estou à disposição para esclarecimentos.",
    "Pessoal, após a última atualização, percebemos que algumas páginas do site estão com erro. Podem verificar isso o mais rápido possível? Precisamos corrigir antes da próxima campanha ir ao ar.",
    "VOCÊ FOI SELECIONADO! Ganhe um iPhone 15 grátis! Parabéns! Você foi escolhido para ganhar um iPhone 15 de graça. Para receber seu prêmio, basta clicar no link abaixo e preencher o formulário. Mas corra, a oferta expira em poucas horas!",
    "Oi equipe, gostaria de compartilhar alguns pontos de melhoria que identificamos na última campanha. Podemos marcar uma reunião para discutir e otimizar os próximos passos?"
]

In [4]:
from enum import Enum
from langchain.pydantic_v1 import BaseModel, Field

class SetorEnum(str, Enum):
    """Enumeração dos setores de atendimento"""
    suporte = "suporte"
    financeiro = "financeiro"
    comercial = "comercial"
    atendimento = "atendimento"
    vendas = "vendas"
    span = "span"

class DirecionaSetorAtendimento(BaseModel):
    """Informa o setor de atendimento de acordo com o nome do cliente"""
    setor: SetorEnum

In [5]:
from langchain_core.utils.function_calling import  convert_to_openai_function

tool_direcionamento = convert_to_openai_function(DirecionaSetorAtendimento)

for email in emails:
    resposta = chat.invoke(email, functions=[tool_direcionamento])
    print(resposta)

content='' additional_kwargs={'function_call': {'arguments': '{"setor":"vendas"}', 'name': 'DirecionaSetorAtendimento'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None} id='run-aac32975-ad97-4892-99e7-e76e6372f941-0' usage_metadata={'input_tokens': 150, 'output_tokens': 23, 'total_tokens': 173, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='' additional_kwargs={'function_call': {'arguments': '{"setor":"atendimento"}', 'name': 'DirecionaSetorAtendimento'}, 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 23,

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente que categoriza emails."),
    ("user", "{input}")
])

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = prompt | chat.bind(functions=[tool_direcionamento], function_call={"name": "DirecionaSetorAtendimento"}) | JsonOutputFunctionsParser()

email = emails[5]
duvidas = chain.invoke({"input": email})

print("Email: ", email)
print("Resposta: ", duvidas)

Email:  Pessoal, após a última atualização, percebemos que algumas páginas do site estão com erro. Podem verificar isso o mais rápido possível? Precisamos corrigir antes da próxima campanha ir ao ar.
Resposta:  {'setor': 'suporte'}
